In [1]:
import pandas as pd
df= pd.read_csv("EDA_dataset.csv")
df.head()

,Unnamed: 0,smiles,graph,label,mol,mol_wt,num_atoms,num_rings,smiles_len,logp
0,0,CC(O)(P(=O)(O)O)P(=O)(O)O,<deepchem.feat.mol_graphs.ConvMol object at 0x...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,206.027,11,0,25,-0.9922
1,1,CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C,<deepchem.feat.mol_graphs.ConvMol object at 0x...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,290.444,20,0,36,4.8172
2,2,OC[C@H](O)[C@@H](O)[C@H](O)CO,<deepchem.feat.mol_graphs.ConvMol object at 0x...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,152.146,10,0,29,-2.9463
3,3,CCCCCCCC(=O)[O-].CCCCCCCC(=O)[O-].[Zn+2],<deepchem.feat.mol_graphs.ConvMol object at 0x...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,351.802,21,0,40,2.1911
4,4,CC(C)COC(=O)C(C)C,<deepchem.feat.mol_graphs.ConvMol object at 0x...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,144.214,10,0,17,1.8416


In [2]:
tox21_tasks=['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']
task_name= 'SR-MMP'
index_test= tox21_tasks.index(task_name)
df['labels_SP_test']=df['label'].apply(lambda x:x[index_test]).astype(int)

In [3]:
import numpy as np, pandas as pd, math, random, os
from rdkit import Chem
from rdkit.Chem import AllChem
from collections import defaultdict

SEED = 40
np.random.seed(SEED); random.seed(SEED)


# Keep (smiles, label) and canonicalize
def canonical_smiles(s):
    m = Chem.MolFromSmiles(s)
    return Chem.MolToSmiles(m) if m else None

dfc = df.dropna().copy()
dfc['smiles'] = dfc['smiles'].apply(canonical_smiles)
dfc.drop(columns=['label','Unnamed: 0'],inplace=True)
dfc = dfc.dropna(subset=['smiles']).drop_duplicates(subset=['smiles']).reset_index(drop=True)
dfc.head()

[12:00:51] WARNING: not removing hydrogen atom without neighbors


,smiles,graph,mol,mol_wt,num_atoms,num_rings,smiles_len,logp,labels_SP_test
0,CC(O)(P(=O)(O)O)P(=O)(O)O,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,206.027,11,0,25,-0.9922,0
1,CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,290.444,20,0,36,4.8172,0
2,OC[C@@H](O)[C@H](O)[C@@H](O)CO,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,152.146,10,0,29,-2.9463,0
3,CCCCCCCC(=O)[O-].CCCCCCCC(=O)[O-].[Zn+2],<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,351.802,21,0,40,2.1911,0
4,CC(C)COC(=O)C(C)C,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027747E...,144.214,10,0,17,1.8416,0


In [4]:
dfc['labels_SP_test'].astype(int).sum()

np.int64(208)

In [5]:
from rdkit.Chem.Scaffolds import MurckoScaffold

def scaffold(smiles):
    m = Chem.MolFromSmiles(smiles)
    if m is None: return None
    scaf = MurckoScaffold.GetScaffoldForMol(m)
    return Chem.MolToSmiles(scaf) if scaf else None

dfc['scaffold'] = dfc['smiles'].apply(scaffold)
# bucket scaffolds
scaf2idxs = defaultdict(list)
for i, s in enumerate(dfc['scaffold']):
    scaf2idxs[s].append(i)

[12:00:53] WARNING: not removing hydrogen atom without neighbors


In [6]:
dfc['scaffold'] = dfc['smiles'].apply(scaffold)
# bucket scaffolds
scaf2idxs = defaultdict(list)
for i, s in enumerate(dfc['scaffold']):
    scaf2idxs[s].append(i)

# sort buckets by descending size to pack splits stably
buckets = sorted(scaf2idxs.values(), key=lambda x: -len(x))
N = len(dfc); tr_idx, va_idx, te_idx = [], [], []
tr_budget, va_budget = int(0.8*N), int(0.1*N)

for b in buckets:
    if len(tr_idx) + len(b) <= tr_budget:
        tr_idx += b
    elif len(va_idx) + len(b) <= va_budget:
        va_idx += b
    else:
        te_idx += b

tr, va, te = dfc.iloc[tr_idx].reset_index(drop=True), dfc.iloc[va_idx].reset_index(drop=True), dfc.iloc[te_idx].reset_index(drop=True)


[12:00:55] WARNING: not removing hydrogen atom without neighbors


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MolToSmiles

# Data augmentation function
def randomize_smiles(smiles, num_aug=5):
    """
    Generates a list of randomized SMILES strings from a single SMILES string.
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:  # Handle invalid SMILES strings
        return [None] * num_aug
    return [MolToSmiles(mol, doRandom=True) for _ in range(num_aug)]

positive_class_df = tr[tr['labels_SP_test'] == 1].copy()

# Apply the function to the positive class and explode
augmented_df = positive_class_df.assign(
    smiles=positive_class_df['smiles'].apply(randomize_smiles, num_aug=5)
).explode('smiles')

# Combine the augmented positive class with the original negative class
negative_class_df = tr[tr['labels_SP_test'] == 0]
tr_df = pd.concat([negative_class_df, augmented_df], ignore_index=True)

# Shuffle the final DataFrame
tr = tr_df.sample(frac=1).reset_index(drop=True)

In [8]:
def label_summary(name, y):
    import numpy as np
    u, c = np.unique(y, return_counts=True)
    print(f"{name}: classes={dict(zip(u.tolist(), c.tolist()))}, "
          f"pos_frac={c[u.tolist().index(1)]/c.sum():.4f}" if 1 in u else
          f"{name}: classes={dict(zip(u.tolist(), c.tolist()))}, pos_frac=0.0")

# After you’ve built datasets:
y_tr = tr['labels_SP_test'].astype(int).values
y_va = va['labels_SP_test'].astype(int).values
y_te = te['labels_SP_test'].astype(int).values

label_summary("TRAIN", y_tr)
label_summary("VAL",   y_va)
label_summary("TEST",  y_te)


TRAIN: classes={0: 4870, 1: 2720}, pos_frac=0.3584
VAL: classes={0: 588, 1: 37}, pos_frac=0.0592
TEST: classes={0: 592, 1: 35}, pos_frac=0.0558


In [9]:
# ==== ECFP4 fingerprints ====
from rdkit.Chem import DataStructs

def ecfp4(smiles, n_bits=2048, radius=2):
    m = Chem.MolFromSmiles(smiles)
    if m is None: return None
    bv = AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(bv, arr)
    return arr

def featurize(df_):
    X = []
    keep = []
    for i, s in enumerate(df_['smiles']):
        fp = ecfp4(s)
        if fp is None: continue
        X.append(fp); keep.append(i)
    X = np.stack(X)
    y = df_.iloc[keep]['labels_SP_test'].astype(int).values
    return X, y, df_.iloc[keep].reset_index(drop=True)

X_tr, y_tr, tr = featurize(tr)
X_va, y_va, va = featurize(va)
X_te, y_te, te = featurize(te)

[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerator
[12:00:56] DEPRECATION WARNING: please use MorganGenerat

In [10]:
te.head()

,smiles,graph,mol,mol_wt,num_atoms,num_rings,smiles_len,logp,labels_SP_test,scaffold
0,c1ccc2c(c1)-c1cccc3cccc-2c13,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027737D...,202.256,16,4,28,4.4872,0,c1ccc2c(c1)-c1cccc3cccc-2c13
1,Nc1ccc2c3c(cccc13)-c1ccccc1-2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027737D...,217.271,17,4,29,4.0694,0,c1ccc2c(c1)-c1cccc3cccc-2c13
2,C[C@]12CCC(=O)C[C@@H]1CC[C@@H]1[C@@H]2CC[C@]2(...,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027737D...,290.447,21,4,64,3.9591,0,O=C1CCC2[C@@H](CC[C@H]3[C@@H]4CCCC4CC[C@H]23)C1
3,C[C@]12CCC(=O)C[C@@H]1CC[C@@H]1[C@@H]2CC[C@@]2...,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027737D...,304.474,22,4,66,4.3492,0,O=C1CCC2[C@@H](CC[C@H]3[C@@H]4CCCC4CC[C@H]23)C1
4,CCCN(CCC)CCc1cccc2c1CC(=O)N2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,<rdkit.Chem.rdchem.Mol object at 0x0000027737D...,260.381,19,2,28,2.8457,0,O=C1Cc2ccccc2N1


In [11]:
# ==== Handle imbalance: compute class weight for pos ====
neg = (y_tr==0).sum(); pos = (y_tr==1).sum()
pos_weight = neg / max(pos, 1)

In [12]:
# ==== Baselines: Logistic Regression + RandomForest ====
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, matthews_corrcoef, precision_recall_curve,recall_score

# Logistic Regression with class_weight
lr = LogisticRegression(max_iter=2000, class_weight={0:1.0, 1:pos_weight}, n_jobs=-1)
lr.fit(X_tr, y_tr)
p_va = lr.predict_proba(X_va)[:,1]; p_te = lr.predict_proba(X_te)[:,1]

# Tune threshold on validation by F1 or best F1-PR tradeoff
prec, rec, thr = precision_recall_curve(y_va, p_va)
f1s = 2*prec*rec/(prec+rec+1e-9)
best_idx = np.argmax(f1s)
best_thr = thr[max(best_idx-1,0)] if len(thr)>0 else 0.5

def report(split, y, p, thr=0.5):
    return {
        f'{split}_ROC_AUC': roc_auc_score(y, p),
        f'{split}_Recal': recall_score(y,(p>=thr).astype(int)),
        f'{split}_PR_AUC': average_precision_score(y, p),
        f'{split}_F1@thr': f1_score(y, (p>=thr).astype(int)),
        f'{split}_MCC@thr': matthews_corrcoef(y, (p>=thr).astype(int))
    }

lr_metrics = {}
lr_metrics.update(report('val', y_va, p_va, best_thr))
lr_metrics.update(report('test', y_te, p_te, best_thr))

# RandomForest (simple config)
rf = RandomForestClassifier(n_estimators=600, max_depth=None, class_weight={0:1.0, 1:pos_weight}, n_jobs=-1, random_state=SEED)
rf.fit(X_tr, y_tr)
prf_va = rf.predict_proba(X_va)[:,1]; prf_te = rf.predict_proba(X_te)[:,1]
rf_metrics = {}
rf_metrics.update(report('val', y_va, prf_va, best_thr))
rf_metrics.update(report('test', y_te, prf_te, best_thr))

print("LR:", lr_metrics)
print("RF:", rf_metrics)

LR: {'val_ROC_AUC': 0.6970950542379114, 'val_Recal': 0.2702702702702703, 'val_PR_AUC': 0.16560704073416707, 'val_F1@thr': 0.2564102564102564, 'val_MCC@thr': 0.20737128912810227, 'test_ROC_AUC': 0.586003861003861, 'test_Recal': 0.02857142857142857, 'test_PR_AUC': 0.06687973650343362, 'test_F1@thr': 0.024390243902439025, 'test_MCC@thr': -0.04283413259423384}
RF: {'val_ROC_AUC': 0.725202243059386, 'val_Recal': 0.05405405405405406, 'val_PR_AUC': 0.15516249310690197, 'val_F1@thr': 0.0975609756097561, 'val_MCC@thr': 0.14990505348225183, 'test_ROC_AUC': 0.5943774131274131, 'test_Recal': 0.0, 'test_PR_AUC': 0.08421499769645038, 'test_F1@thr': 0.0, 'test_MCC@thr': -0.016859376189188185}


In [13]:
# ==== PyTorch Geometric dataset builder ====
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import WeightedRandomSampler
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, global_mean_pool, GlobalAttention

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
# Atom features (basic but effective)
def atom_features(atom):
    return np.array([
        atom.GetAtomicNum(),                    # Z
        atom.GetTotalDegree(),                  # degree
        atom.GetFormalCharge(),                 # charge
        int(atom.GetTotalNumHs()),              # H count
        int(atom.GetIsAromatic())               # aromatic
    ], dtype=np.int64)

# Edge/bond features
def bond_features(bond):
    if bond is None: # self-loop
        bt, ar, conj = 0, 0, 0
    else:
        bt = int(bond.GetBondTypeAsDouble())    # 1,2,3,arom approx
        ar = int(bond.GetIsAromatic())
        conj = int(bond.GetIsConjugated())
    return np.array([bt, ar, conj], dtype=np.int64)

def mol_to_pyg(smiles, y):
    m = Chem.MolFromSmiles(smiles)
    if m is None: return None
    Chem.Kekulize(m, clearAromaticFlags=False)
    # nodes
    x = np.vstack([atom_features(a) for a in m.GetAtoms()]).astype(np.int64)
    # edges (undirected)
    ei_src, ei_dst, eattr = [], [], []
    for b in m.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bf = bond_features(b)
        ei_src += [i, j]; ei_dst += [j, i]
        eattr += [bf, bf]
    if len(ei_src)==0:
        # handle single-atom molecules: add self-loop
        ei_src=[0]; ei_dst=[0]; eattr=[bond_features(None)]
    edge_index = torch.tensor([ei_src, ei_dst], dtype=torch.long)
    edge_attr = torch.tensor(np.vstack(eattr), dtype=torch.long)
    x = torch.tensor(x, dtype=torch.long)
    y = torch.tensor([y], dtype=torch.float32)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

class ToxDataset(InMemoryDataset):
    def __init__(self, df_, transform=None):
        super().__init__('.', transform)
        data_list = []
        for s, y in zip(df_['smiles'], df_['labels_SP_test'].astype(int)):
            g = mol_to_pyg(s, y)
            if g is not None: data_list.append(g)
        self.data, self.slices = self.collate(data_list)

    def num_node_features(self):
        return int(self.data.x.max().item())+1  # categorical encoding workaround (we'll embed)

# Build datasets
ds_tr = ToxDataset(tr)
ds_va = ToxDataset(va)
ds_te = ToxDataset(te)

[12:01:47] WARNING: not removing hydrogen atom without neighbors


In [ ]:
# ==== Model: GINE + attention readout ====

class GINEBlock(nn.Module):
    def __init__(self, hidden, edge_emb_dim=8):
        super().__init__()
        self.edge_emb = nn.Embedding(16, edge_emb_dim)  # small shared embedding via hashing
        self.mlp = nn.Sequential(
            nn.Linear(hidden, hidden), nn.ReLU(),
            nn.Linear(hidden, hidden)
        )
        self.conv = GINEConv(self.mlp, train_eps=True, edge_dim=edge_emb_dim)

    def forward(self, x, edge_index, edge_attr):
        # compress 3-dim edge_attr to single index via hashing (bounded)
        h = (edge_attr[:,0] + 3*edge_attr[:,1] + 7*edge_attr[:,2]) % 16
        e = self.edge_emb(h)
        return self.conv(x, edge_index, e)

class GNN(nn.Module):
    def __init__(self, hidden=64, node_vocab_sizes=(120,10,7,5,2), emb_dims=(64,16,8,8,4)):
        super().__init__()
        # 5 categorical fields per node: Z, degree, charge, Hs, aromatic
        self.node_embs = nn.ModuleList([
            nn.Embedding(v, d) for v, d in zip(node_vocab_sizes, emb_dims)
        ])
        node_dim = sum(emb_dims)
        self.proj = nn.Linear(node_dim, hidden)

        self.g1 = GINEBlock(hidden)
        self.g2 = GINEBlock(hidden)
        self.g3 = GINEBlock(hidden)
        self.bn1 = torch.nn.BatchNorm1d(hidden)

        # attention readout
        self.gate_nn = nn.Sequential(nn.Linear(hidden, 1))
        self.readout = GlobalAttention(self.gate_nn)   # attention-weighted sum

        self.head = nn.Sequential(
            nn.Linear(hidden, 128), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)  # logit
        )

    def forward(self, data):
        x_cat = []
        # split x into fields
        Z, deg, chg, Hs, aro = data.x[:,0], data.x[:,1], data.x[:,2], data.x[:,3], data.x[:,4]
        for emb, feat in zip(self.node_embs, [Z,deg,chg,Hs,aro]):
            x_cat.append(emb(feat.clamp(min=0, max=emb.num_embeddings-1)))
        x = torch.cat(x_cat, dim=-1)
        x = self.proj(x)

        x = F.relu(self.g1(x, data.edge_index, data.edge_attr)))
        #x = F.relu(self.g2(x, data.edge_index, data.edge_attr))
        x = self.bn1(F.relu(self.g3(x, data.edge_index, data.edge_attr)))

        # batch handling
        batch = getattr(data, 'batch', torch.zeros(x.size(0), dtype=torch.long, device=x.device))
        hg = self.readout(x, batch)

        logit = self.head(hg).squeeze(-1)
        return logit


In [26]:
from torch_geometric.utils import dropout_edge

from torch_geometric.utils import dropout_edge

def apply_dropedge(data, p=0.2):
    # Drop edges randomly while preserving graph structure
    edge_index, edge_attr = data.edge_index, data.edge_attr
    
    # dropout_edge returns a mask (which edges to keep)
    edge_index, edge_mask = dropout_edge(edge_index, p=p, training=True)

    if edge_attr is not None:
        edge_attr = edge_attr[edge_mask]  # keep edge features consistent

    # Build a new data object OR modify in place
    data.edge_index = edge_index
    data.edge_attr = edge_attr
    return data


def mask_node_features(x, mask_prob=0.15):
    mask = torch.rand(x.size()) < mask_prob
    x = x.clone()
    x[mask] = 0
    return x

In [27]:
# ==== Loaders + imbalance handling ====
BATCH=64
# Weighted sampler (optional). Often pos minority → sample more
class_counts = np.bincount(y_tr)
w = np.zeros_like(y_tr, dtype=np.float32)
w[y_tr==0] = class_counts[1]/(class_counts[0]+1e-9)
w[y_tr==1] = 1.0
sampler = WeightedRandomSampler(weights=w, num_samples=len(w), replacement=True)

train_loader = DataLoader(ds_tr, batch_size=BATCH, sampler=sampler)
val_loader   = DataLoader(ds_va, batch_size=BATCH, shuffle=False)
test_loader  = DataLoader(ds_te, batch_size=BATCH, shuffle=False)

# ==== Train ====
from sklearn.metrics import roc_auc_score, average_precision_score

model = GNN(hidden=128).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# BCE with pos_weight
pos = (y_tr==1).sum(); neg = (y_tr==0).sum()
pos_weight_t = torch.tensor([neg/max(pos,1)], dtype=torch.float32, device=device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_t)

def run_epoch(loader, train=False):
    if train: model.train()
    else: model.eval()
    losses, y_true, y_prob = [], [], []
    for batch in loader:
        batch = batch.to(device)
        #print(batch.edge_attr.shape)
        # if train:
        #     batch = apply_dropedge(batch, p=0.2)
        logit = model(batch)
        loss = criterion(logit, batch.y)
        if train:
            optimizer.zero_grad(); loss.backward(); optimizer.step()
        losses.append(loss.detach().item())
        y_true.append(batch.y.detach().cpu().numpy())
        y_prob.append(torch.sigmoid(logit).detach().cpu().numpy())
    y_true = np.concatenate(y_true); y_prob = np.concatenate(y_prob)
    roc = roc_auc_score(y_true, y_prob) if len(np.unique(y_true))>1 else float('nan')
    pr  = average_precision_score(y_true, y_prob) if len(np.unique(y_true))>1 else float('nan')
    return np.mean(losses), roc, pr

best_pr, best_state = -1, None
patience, bad = 20, 0

for epoch in range(1, 201):
    tr_loss, tr_roc, tr_pr = run_epoch(train_loader, train=True)
    va_loss, va_roc, va_pr = run_epoch(val_loader, train=False)
    if va_pr > best_pr:
        best_pr, bad = va_pr, 0
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
    else:
        bad += 1
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} | TR loss {tr_loss:.4f} ROC {tr_roc:.3f} PR {tr_pr:.3f} || VA loss {va_loss:.4f} ROC {va_roc:.3f} PR {va_pr:.3f}")
    if bad >= patience: break

model.load_state_dict(best_state)
te_loss, te_roc, te_pr = run_epoch(test_loader, train=False)
print(f"TEST  ROC-AUC: {te_roc:.4f} | PR-AUC: {te_pr:.4f}")

c:\Projects\QChem\venv\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch 010 | TR loss 0.3034 ROC 0.954 PR 0.939 || VA loss 1.2031 ROC 0.622 PR 0.122
Epoch 020 | TR loss 0.2460 ROC 0.966 PR 0.953 || VA loss 1.1180 ROC 0.641 PR 0.124
Epoch 030 | TR loss 0.1920 ROC 0.977 PR 0.970 || VA loss 1.5339 ROC 0.606 PR 0.137
TEST  ROC-AUC: 0.6642 | PR-AUC: 0.1480
